# Habitat suitability for Red Pandas -- Dataset Synthesis

# Synthetic Habitat Suitability Dataset Creation Summary

A synthetic dataset has been created to model the habitat suitability for pandas. The dataset comprises 100,000 samples, each with the following variables:

- `Altitude`: Generated from a uniform distribution ranging from 1200 to 3400 meters.
- `Distance_from_Human_Paths`: Generated from an exponential distribution, with a mean distance of 500 meters.
- `Livestock_Density`: Generated from a gamma distribution to represent skewed data, with a shape parameter of 2 and a scale of 0.5.
- `Vegetation_Diversity_Index`: Generated from a uniform distribution between 0 and 1.
- `Water_Source_Availability`: Binary variable generated with a preference for '1' (70%) over '0' (30%).
- `Human_Disturbance_Index`: Generated from a uniform distribution between 0 and 1.
- `Slope`: Generated from a uniform distribution between 0 and 30 degrees.
- `Annual_Rainfall`: Generated from a normal distribution centered at 1500 mm with a standard deviation of 250 mm.

Correlations and noise have been introduced to reflect ecological relationships:

1. **Altitude and Bamboo Coverage**: A positive linear relationship with 10% noise added.
2. **Livestock Density and Vegetation Diversity Index**: An inverse relationship modified by the exponential of negative livestock density, with 5% noise added.
3. **Water Source Availability and Annual Rainfall**: A conditional relationship where higher rainfall increases the likelihood of water availability, with 10% noise introduced by inverting the availability status.
4. **Distance from Human Paths and Human Disturbance Index**: An inverse relationship where greater distance from paths lowers the disturbance index, with 10% noise added.
5. **Slope and Bamboo Coverage**: A conditional relationship where slopes under 20 degrees do not affect bamboo coverage, while steeper slopes reduce coverage by half, with 10% noise added.

The `Suitable` variable is defined based on certain thresholds and conditions:

- Altitude between 1500 and 3000 meters.
- Distance from human paths greater than 500 meters.
- Livestock density lower than 5.
- Bamboo coverage above 20%.
- Vegetation diversity index above 0.3.
- Water source available.
- Human disturbance index below 0.5.
- Slope less than 20 degrees.
- Annual rainfall greater than 1000 mm.

Each of these conditions must be met for a habitat to be considered suitable (`Suitable` = 1).

Lastly, `Mean_Annual_Temperature` is derived from altitude with a simple linear relationship and added noise, to reflect the decrease in temperature with increasing altitude.

The dataset synthesis process includes random noise to simulate natural variability and to prevent perfect correlations, aiming to create a realistic dataset for training predictive models.


**Features values generation**

In [97]:
import pandas as pd
import numpy as np

# Set the random seed for reproducibility
np.random.seed(42)

# Number of samples in the dataset
n_samples = 100000

# Generate the synthetic data
data = pd.DataFrame({
    'Altitude': np.random.uniform(1200, 3400, n_samples),  # Uniform distribution
    'Distance_from_Human_Paths': np.random.exponential(500, n_samples),  # Exponential distribution
    'Livestock_Density': np.random.gamma(2, 0.5, n_samples),  # Gamma distribution for skewed data
    'Vegetation_Diversity_Index': np.random.uniform(0, 1, n_samples),  # Uniform distribution

    # This one is not good, needs to be corrected
    'Water_Source_Availability': np.random.choice([0, 1], n_samples, p=[0.3, 0.7]),  # Binary, with more 1s
    'Human_Disturbance_Index': np.random.uniform(0, 1, n_samples),  # Uniform distribution
    'Slope': np.random.uniform(0, 30, n_samples),  # Uniform distribution
    'Annual_Rainfall': np.random.normal(1500, 250, n_samples)  # Normal distribution
})

# Add noise function
def add_noise(factor, size):
    return np.random.normal(1, factor, size)

# Correlation 1: Altitude and Bamboo Coverage (with noise)
data['Bamboo_Coverage'] = np.interp(data['Altitude'], [1200, 3400], [0, 100])
data['Bamboo_Coverage'] *= add_noise(0.1, n_samples)  # Adding 10% noise

# Correlation 2: Livestock Density and Vegetation Diversity Index (with noise)
data['Vegetation_Diversity_Index'] *= np.exp(-data['Livestock_Density'])
data['Vegetation_Diversity_Index'] *= add_noise(0.05, n_samples)  # Adding 5% noise

# Correlation 3: Water Source Availability and Annual Rainfall (with noise)
data['Water_Source_Availability'] = np.where(data['Annual_Rainfall'] > data['Annual_Rainfall'].mean(), 1, 0)
data['Water_Source_Availability'] = np.where(np.random.rand(n_samples) < 0.1, 1 - data['Water_Source_Availability'], data['Water_Source_Availability'])  # Inverting 10% to add noise

# Correlation 4: Distance from Human Paths and Human Disturbance Index (with noise)
data['Human_Disturbance_Index'] = np.interp(data['Distance_from_Human_Paths'], [0, max(data['Distance_from_Human_Paths'])], [1, 0])
data['Human_Disturbance_Index'] *= add_noise(0.1, n_samples)  # Adding 10% noise

# Correlation 5: Slope and Bamboo Coverage (with noise)
data['Bamboo_Coverage'] *= np.where(data['Slope'] < 20, 1, 0.5)  # Reduce bamboo coverage on steep slopes
data['Bamboo_Coverage'] *= add_noise(0.1, n_samples)  # Adding 10% noise

# Mean Annual Temperature is derived with some noise, considering its correlation with Altitude
data['Mean_Annual_Temperature'] = 20 - (data['Altitude'] / 200)  # Simplified linear relation
data['Mean_Annual_Temperature'] += np.random.normal(0, 1, n_samples)  # Adding noise

# Output the first few rows of the dataset
data.head()

,Altitude,Distance_from_Human_Paths,Livestock_Density,Vegetation_Diversity_Index,Water_Source_Availability,Human_Disturbance_Index,Slope,Annual_Rainfall,Bamboo_Coverage,Mean_Annual_Temperature
0,2023.988261,434.678578,0.634095,0.319653,0,1.025552,1.902428,1364.685583,30.270545,9.912647
1,3291.571474,374.299977,0.132676,0.604097,1,1.032245,16.484248,1616.727901,71.382519,1.701986
2,2810.386672,216.189749,0.477984,0.157106,0,0.963946,29.466434,1373.229653,31.969303,6.258988
3,2517.048665,339.831903,0.386706,0.560503,1,1.142016,9.966878,1747.708413,50.076009,7.423174
4,1543.241009,227.141237,0.654869,0.349722,1,1.050573,10.647234,1500.073131,17.449099,12.331676


In [87]:
data.Suitable.value_counts()

0    99975
1       25
Name: Suitable, dtype: int64

**Correlation Matrix**

In [88]:
data.corr()

,Altitude,Distance_from_Human_Paths,Livestock_Density,Vegetation_Diversity_Index,Water_Source_Availability,Human_Disturbance_Index,Slope,Annual_Rainfall,Bamboo_Coverage,Suitable,Mean_Annual_Temperature
Altitude,1.000000,-0.001083,0.001307,0.001045,-0.003451,0.001926,0.004291,0.000666,0.843007,0.003605,-0.953462
Distance_from_Human_Paths,-0.001083,1.000000,0.000990,-0.000224,-0.000640,-0.689589,-0.004143,-0.002787,-0.001336,0.080682,-0.000189
Livestock_Density,0.001307,0.000990,1.000000,-0.567181,0.000030,0.001556,0.001104,-0.003032,0.002092,-0.012718,-0.000831
Vegetation_Diversity_Index,0.001045,-0.000224,-0.567181,1.000000,0.002981,-0.000260,-0.002427,0.000985,0.000460,0.021991,-0.001660
Water_Source_Availability,-0.003451,-0.000640,0.000030,0.002981,1.000000,-0.001188,-0.002194,0.637011,-0.005819,0.015768,0.003782
Human_Disturbance_Index,0.001926,-0.689589,0.001556,-0.000260,-0.001188,1.000000,-0.000112,0.000138,0.003379,-0.060546,-0.001335
Slope,0.004291,-0.004143,0.001104,-0.002427,-0.002194,-0.000112,1.000000,0.000365,-0.334804,-0.009101,-0.003838
Annual_Rainfall,0.000666,-0.002787,-0.003032,0.000985,0.637011,0.000138,0.000365,1.000000,-0.001680,0.005696,-0.000014
Bamboo_Coverage,0.843007,-0.001336,0.002092,0.000460,-0.005819,0.003379,-0.334804,-0.001680,1.000000,0.007994,-0.804516
Suitable,0.003605,0.080682,-0.012718,0.021991,0.015768,-0.060546,-0.009101,0.005696,0.007994,1.000000,-0.002320


**'Suitable' column estimation -- KMeans**

Artificially manipulated to get 30% of environments as suitable.

In [89]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

# Assuming `data` is your DataFrame with all the features but without the 'Suitable' column

# Normalize the data for clustering
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Use KMeans clustering
kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(data_scaled)

# Add the cluster information to the original data
data['Cluster'] = clusters

# Identify the centroids for each cluster
centroids = kmeans.cluster_centers_

# Calculate the distance of each point to the centroids
distances = kmeans.transform(data_scaled)

# Get the distances to the "farthest" centroid (assuming cluster 0 is 'unsuitable', cluster 1 is 'suitable')
data['Distance_to_Unsuitable'] = distances[:, 0]
data['Distance_to_Suitable'] = distances[:, 1]

# Determine the 30th percentile threshold for the distance to the 'suitable' centroid
threshold = np.percentile(data['Distance_to_Suitable'], 70)  # since we want the closest 30%

# Label the data based on the threshold
data['Suitable'] = (data['Distance_to_Suitable'] <= threshold).astype(int)

# Now, 30% of your data should be labeled as suitable
print(data['Suitable'].value_counts(normalize=True))  # Check the proportion of '1's and '0's

# Drop the columns not needed anymore
data.drop(['Cluster', 'Distance_to_Unsuitable', 'Distance_to_Suitable'], axis=1, inplace=True)

1    0.7
0    0.3
Name: Suitable, dtype: float64


In [90]:
data.Suitable.value_counts()

1    70000
0    30000
Name: Suitable, dtype: int64

In [91]:
data.to_csv('data/habitat-suitability.csv', index=False)

**Baseline Logistic Regression model**

In [92]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Step 1: Prepare the data
# Features matrix 'X' and target variable 'y'
X = data.drop('Suitable', axis=1)
y = data['Suitable']

# Step 2: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 3: Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Initialize and train the Logistic Regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train_scaled, y_train)

# Step 5: Predictions and evaluation
y_pred = log_reg.predict(X_test_scaled)

# Model evaluation
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.88

Confusion Matrix:
 [[ 6912  2027]
 [ 1565 19496]]

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.77      0.79      8939
           1       0.91      0.93      0.92     21061

    accuracy                           0.88     30000
   macro avg       0.86      0.85      0.85     30000
weighted avg       0.88      0.88      0.88     30000



# Transportation Network Structure Dataset Synthesis 

In [93]:
import random 

def generate_optimization_dataset(n_origins, n_transshipments, n_destinations, total_supply,
                                  min_trans_cost = 1, max_trans_cost = 100):
    
    X = n_origins
    Y = n_destinations
    Z = n_transshipments
    
    ## Transportation Costs
    # Origin to transshipment
    origin_to_transshipment_cost = {}
    for i in range(X):
        for k in range(Z):
            key = f"{i+1}-{k+1}"
            cost = random.randint(min_trans_cost, max_trans_cost)  # Random uniform
            origin_to_transshipment_cost[key] = cost

    # Transshipment to destination
    transshipment_to_destination_cost = {}
    for k in range(Z):
        for j in range(Y):
            key = f"{k+1}-{j+1}"
            cost = random.randint(min_trans_cost, max_trans_cost)  # Random uniform
            transshipment_to_destination_cost[key] = cost


    ## Capacities
    # Transhipment capacity

    min_capacity_hub = total_supply // Z # To be refined
    max_capacity_hub = total_supply // Z + 1000

    transshipment_capacity = {}
    for k in range(Z):
        capacity = random.randint(min_capacity_hub, max_capacity_hub)  # Random capacity between 1000 and 5000
        transshipment_capacity[f"{k+1}"] = capacity

    # Destination capacity

    min_capacity_destination = total_supply // Y # To be refined
    max_capacity_destination = total_supply
    destination_capacity = {}
    for j in range(Y):
        capacity = random.randint(min_capacity_destination, max_capacity_destination)
        destination_capacity[f"{j+1}"] = capacity


    ## Supply of origin nodes
    origin_supply = {}

    proportions = np.random.dirichlet(np.ones(X),size=1)

    # Distribute the demand across all origin supply nodes
    for i in range(X):
        origin_supply[f"{i+1}"] = int(proportions[0][i] * total_supply)
    
    return [origin_to_transshipment_cost, transshipment_to_destination_cost, transshipment_capacity, destination_capacity, origin_supply]


In [94]:
network = generate_optimization_dataset(n_origins=30, n_transshipments=10, n_destinations=10, total_supply=10000)

In [95]:
origin_to_transshipment_cost = network[0]
origin_to_transshipment_cost_df = pd.DataFrame(list(origin_to_transshipment_cost.items()), columns=['Key', 'Value'])
origin_to_transshipment_cost_df['Origin'] = origin_to_transshipment_cost_df['Key'].str.split('-').str[0]
origin_to_transshipment_cost_df['Hub'] = origin_to_transshipment_cost_df['Key'].str.split('-').str[1]
origin_to_transshipment_cost_df = origin_to_transshipment_cost_df.drop('Key', axis=1)


transshipment_to_destination_cost = network[1]
transshipment_to_destination_cost_df = pd.DataFrame(list(transshipment_to_destination_cost.items()), columns=['Key', 'Value'])
transshipment_to_destination_cost_df['Hub'] = transshipment_to_destination_cost_df['Key'].str.split('-').str[0]
transshipment_to_destination_cost_df['Destination'] = transshipment_to_destination_cost_df['Key'].str.split('-').str[1]
transshipment_to_destination_cost_df = transshipment_to_destination_cost_df.drop('Key', axis=1)

transshipment_capacity = network[2] 
transshipment_capacity_df = pd.DataFrame(list(transshipment_capacity.items()), columns=['Hub', 'Value'])

destination_capacity = network[3]
destination_capacity_df = pd.DataFrame(list(destination_capacity.items()), columns=['Destination', 'Value'])

origin_supply = network[4]
origin_supply_df = pd.DataFrame(list(origin_supply.items()), columns=['Origin', 'Value'])

In [96]:
origin_to_transshipment_cost_df.to_csv('data/opti-origin_to_transshipment_cost.csv', index=False)
transshipment_to_destination_cost_df.to_csv('data/opti-transshipment_to_destination_cost.csv', index=False)  
transshipment_capacity_df.to_csv('data/opti-transshipment_capacity.csv', index=False)
destination_capacity_df.to_csv('data/opti-destination_capacity.csv', index=False)
origin_supply_df.to_csv('data/opti-origin_supply.csv', index=False)